# 必要ライブラリの読み込み

In [1]:
#ライブラリ読み込み
import lightgbm as lgb
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import optuna
from tqdm import tqdm
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

# 必要データの読み込み

In [2]:
#データ読み込み

#データ定義
train_input= pd.read_csv('train_0806best.csv')
test_input= pd.read_csv('test_0806best.csv')

In [3]:
category_columns = ['city1',
 'city2',
 'city3',
 '階数',
 '部屋数',
 'jukyo',
 'yoto1',
 'yoto2',
 'josui',
 'gesui',
 'gas',
 'usui',
 'road1_hk',
 'road1_sb',
 'road2_hk',
 'road2_sb',
 'road3_hk',
 'road3_sb',
 'road4_hk',
 'road4_sb',
 'hw_status',
 'toshikuiki1',
 'toshikuiki2',
 'kodochiku',
 'bokachiiki',
 'hokakisei1',
 'hokakisei2',
 'hokakisei3',
 'hokakisei4',
 'kinshijiko',
 'rosen_nm1',
 'eki_nm1',
 'bas_toho1',
 'bastei_nm1',
 'rosen_nm2',
 'eki_nm2',
 'bas_toho2',
 'bastei_nm2',
 'levelplan',
 'road_st',
 'setsudo_hi',
 'setsudo_kj',
 'jigata',
 'hiatari',
 'kobetsu1',
 'kobetsu2',
 'kobetsu3',
 'kobetsu4']

In [4]:
#LightGBMでのカテゴリ扱いをする変数を指定
for column in category_columns:
    train_input[column] = train_input[column].astype('category')
    test_input[column] = test_input[column].astype('category')

# データ整形

In [5]:
#必要な列だけ残す
X = train_input.drop(['id','pj_no','keiyaku_pr'],axis = 1)
X_test = test_input.drop(['id','pj_no'],axis = 1)
y = train_input[['keiyaku_pr']]

Parameter_df =pd.DataFrame()
MAPE_df = pd.DataFrame()
MAPE_train_df =pd.DataFrame()
Submit_df = test_input[['id']].copy(deep = True)
Submit_df['MAPE_sum'] = 0

#グループ分けの定義
groups = train_input['pj_no'].array


#gkf = GroupShuffleSplit(test_size =0.2,random_state=0)
kf = KFold(n_splits= 5,shuffle=True,random_state=0)

i = 0
#for train,valid  in tqdm(gkf.split(X, y,groups=groups)):
for train,valid in tqdm(kf.split(X,y)):
    X_train = X.iloc[train.tolist()]
    X_valid = X.iloc[valid.tolist()]
    y_train = y.iloc[train.tolist()]
    y_valid = y.iloc[valid.tolist()]

#     # 学習データと検証データに分割する
#     #X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size = 0.2,shuffle = True,random_state = i+10)
#     X_train, X_valid, y_train, y_valid = GroupShuffleSplit(X, y,groups=train_input['pj_no'].array,test_size = 0.2,shuffle = True,random_state = i+10)

    # 上記のパラメータでモデルを学習する

    #'''
#     model = LGBMRegressorCV(
#                             n_estimators = 100,
#                             learning_rate = 0.1,
#                             n_trials = 100,
#                             n_iter_no_change=10,
#                             random_state=0,
#                             timeout = 60)
#     #'''

#     #ハイパーパラメータチューニングしない場合はこっち
#     #model = lgb.LGBMRegressor(random_state=10)

#     #学習
#     model.fit(X_train, y_train,categorical_feature=important_categorical_columns)

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    # LightGBM parameters
    params = {
            'task' : 'train',
            'boosting_type' : 'dart',
            'objective' : 'regression',
            'metric' : {'mape'},
            'num_leaves' : 20,
            'learning_rate' :  0.05,
#             'n_estimators' : 3000,
            'feature_fraction' : 0.5,
            'bagging_fraction' : 0.8,
            'bagging_freq': 5,
            'verbose' : 0
#             'random_state':100
    }


    # train
    model = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval)
#                 early_stopping_rounds=1000)

    #選ばれたハイパーパラメータの記録
    #temp_parameter_df = pd.DataFrame(model.best_params_.items()).set_index(0).T
    #Parameter_df = Parameter_df.append(temp_parameter_df )

    # 検証データを予測する
    #y_pred = model.predict(X_valid)
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)

    #検証スコア算出
    y_pred_df = pd.DataFrame(y_pred)
    y_valid = y_valid.reset_index().drop('index',axis=1)
    compared = pd.concat([y_pred_df,y_valid ],axis = 1)
    compared['APE'] = abs((compared[0]-compared['keiyaku_pr'])/compared['keiyaku_pr'])
    
    #検証スコア算出(train)
    y_pred_train_df = pd.DataFrame(y_pred_train)
    y_train = y_train.reset_index().drop('index',axis=1)
    compared_train = pd.concat([y_pred_train_df,y_train ],axis = 1)
    compared_train['APE'] = abs((compared_train[0]-compared_train['keiyaku_pr'])/compared_train['keiyaku_pr'])

    #検証スコアを記録
    temp_dict = {'MAPE':compared['APE'].mean()*100}
    temp_df = pd.DataFrame(temp_dict.items()).set_index(0).T
    MAPE_df = MAPE_df.append(temp_df)

    #検証スコアを記録
    temp_dict_train = {'MAPE':compared_train['APE'].mean()*100}
    temp_train_df = pd.DataFrame(temp_dict_train.items()).set_index(0).T
    MAPE_train_df = MAPE_train_df.append(temp_train_df)
    
    #予測を実行
    #y_test_pred = model.predict(X_test)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_test_pred_df = pd.DataFrame(y_test_pred)

    #提出用のデータフレーム作成
    temp_submit = pd.concat([test_input[['id']],y_test_pred_df],axis=1)
    temp_submit.rename(columns={0:'MAPE_'+str(i)},inplace = True)
    Submit_df = pd.merge(Submit_df,temp_submit,on='id')

    Submit_df['MAPE_sum'] = Submit_df['MAPE_sum'] + Submit_df['MAPE_'+str(i)]
    i = i+1
    
Submit_df['MAPE_mean'] = Submit_df['MAPE_sum']/(i)
    

0it [00:00, ?it/s]/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's mape: 0.210199
[2]	valid_0's mape: 0.2046
[3]	valid_0's mape: 0.198717
[4]	valid_0's mape: 0.193568
[5]	valid_0's mape: 0.188745
[6]	valid_0's mape: 0.183876
[7]	valid_0's mape: 0.178791
[8]	valid_0's mape: 0.180604
[9]	valid_0's mape: 0.175842
[10]	valid_0's mape: 0.171408
[11]	valid_0's mape: 0.167633
[12]	valid_0's mape: 0.168928
[13]	valid_0's mape: 0.164948
[14]	valid_0's mape: 0.161261
[15]	valid_0's mape: 0.157489
[16]	valid_0's mape: 0.1542
[17]	valid_0's mape: 0.151126
[18]	valid_0's mape: 0.148122
[19]	valid_0's mape: 0.145626
[20]	valid_0's mape: 0.142899
[21]	valid_0's mape: 0.143761
[22]	valid_0's mape: 0.14119
[23]	valid_0's mape: 0.138634
[24]	valid_0's mape: 0.136344
[25]	valid_0's mape: 0.134269
[26]	valid_0's mape: 0.132284
[27]	valid_0's mape: 0.130386
[28]	valid_0's mape: 0.131308
[29]	valid_0's mape: 0.129445
[30]	valid_0's mape: 0.12761
[31]	valid_0's mape: 0.127899
[32]	valid_0's mape: 0.126382
[33]	valid_0's mape: 0.124915
[34]	valid_0's mape: 0.12

[269]	valid_0's mape: 0.100965
[270]	valid_0's mape: 0.0995523
[271]	valid_0's mape: 0.10115
[272]	valid_0's mape: 0.099707
[273]	valid_0's mape: 0.0983143
[274]	valid_0's mape: 0.0995838
[275]	valid_0's mape: 0.0982543
[276]	valid_0's mape: 0.099812
[277]	valid_0's mape: 0.0983406
[278]	valid_0's mape: 0.0995484
[279]	valid_0's mape: 0.0980438
[280]	valid_0's mape: 0.0993874
[281]	valid_0's mape: 0.104908
[282]	valid_0's mape: 0.10308
[283]	valid_0's mape: 0.10461
[284]	valid_0's mape: 0.10267
[285]	valid_0's mape: 0.100955
[286]	valid_0's mape: 0.0992871
[287]	valid_0's mape: 0.0979219
[288]	valid_0's mape: 0.0994476
[289]	valid_0's mape: 0.098041
[290]	valid_0's mape: 0.0966773
[291]	valid_0's mape: 0.0954935
[292]	valid_0's mape: 0.096885
[293]	valid_0's mape: 0.098115
[294]	valid_0's mape: 0.0967641
[295]	valid_0's mape: 0.098004
[296]	valid_0's mape: 0.0991925
[297]	valid_0's mape: 0.097789
[298]	valid_0's mape: 0.0989844
[299]	valid_0's mape: 0.0975797
[300]	valid_0's mape: 0.09

[527]	valid_0's mape: 0.0834311
[528]	valid_0's mape: 0.0829193
[529]	valid_0's mape: 0.0824603
[530]	valid_0's mape: 0.0820345
[531]	valid_0's mape: 0.0824672
[532]	valid_0's mape: 0.0820635
[533]	valid_0's mape: 0.0824605
[534]	valid_0's mape: 0.0819984
[535]	valid_0's mape: 0.0815869
[536]	valid_0's mape: 0.0819319
[537]	valid_0's mape: 0.0815944
[538]	valid_0's mape: 0.0813066
[539]	valid_0's mape: 0.0816534
[540]	valid_0's mape: 0.0820375
[541]	valid_0's mape: 0.0825154
[542]	valid_0's mape: 0.0819551
[543]	valid_0's mape: 0.0823467
[544]	valid_0's mape: 0.0828084
[545]	valid_0's mape: 0.0822924
[546]	valid_0's mape: 0.0818699
[547]	valid_0's mape: 0.081533
[548]	valid_0's mape: 0.0811878
[549]	valid_0's mape: 0.0815368
[550]	valid_0's mape: 0.0812055
[551]	valid_0's mape: 0.0808806
[552]	valid_0's mape: 0.0805547
[553]	valid_0's mape: 0.0808719
[554]	valid_0's mape: 0.0812103
[555]	valid_0's mape: 0.0809455
[556]	valid_0's mape: 0.0813225
[557]	valid_0's mape: 0.0816673
[558]	val

[783]	valid_0's mape: 0.0790006
[784]	valid_0's mape: 0.0793077
[785]	valid_0's mape: 0.0789447
[786]	valid_0's mape: 0.0792323
[787]	valid_0's mape: 0.0795255
[788]	valid_0's mape: 0.0798203
[789]	valid_0's mape: 0.0794319
[790]	valid_0's mape: 0.0790759
[791]	valid_0's mape: 0.0787684
[792]	valid_0's mape: 0.0790101
[793]	valid_0's mape: 0.078687
[794]	valid_0's mape: 0.0789576
[795]	valid_0's mape: 0.0786605
[796]	valid_0's mape: 0.0789107
[797]	valid_0's mape: 0.0791994
[798]	valid_0's mape: 0.0794652
[799]	valid_0's mape: 0.0790913
[800]	valid_0's mape: 0.0793785
[801]	valid_0's mape: 0.0790058
[802]	valid_0's mape: 0.0792838
[803]	valid_0's mape: 0.0795382
[804]	valid_0's mape: 0.0798519
[805]	valid_0's mape: 0.0801007
[806]	valid_0's mape: 0.0804063
[807]	valid_0's mape: 0.0799725
[808]	valid_0's mape: 0.0795676
[809]	valid_0's mape: 0.0798443
[810]	valid_0's mape: 0.0794419
[811]	valid_0's mape: 0.0790674
[812]	valid_0's mape: 0.079328
[813]	valid_0's mape: 0.078993
[814]	valid

...

[3027]	valid_0's mape: 0.0682846
[3028]	valid_0's mape: 0.0682734
[3029]	valid_0's mape: 0.0682607
[3030]	valid_0's mape: 0.0682482
[3031]	valid_0's mape: 0.0682595
[3032]	valid_0's mape: 0.068251
[3033]	valid_0's mape: 0.0682529
[3034]	valid_0's mape: 0.0682473
[3035]	valid_0's mape: 0.0682361
[3036]	valid_0's mape: 0.0682236
[3037]	valid_0's mape: 0.0682124
[3038]	valid_0's mape: 0.0682142
[3039]	valid_0's mape: 0.068205
[3040]	valid_0's mape: 0.0682103
[3041]	valid_0's mape: 0.068199
[3042]	valid_0's mape: 0.0681896
[3043]	valid_0's mape: 0.0681936
[3044]	valid_0's mape: 0.0681837
[3045]	valid_0's mape: 0.0681727
[3046]	valid_0's mape: 0.0681964
[3047]	valid_0's mape: 0.0681908
[3048]	valid_0's mape: 0.0681961
[3049]	valid_0's mape: 0.0681859
[3050]	valid_0's mape: 0.0681958
[3051]	valid_0's mape: 0.0682117
[3052]	valid_0's mape: 0.0682207
[3053]	valid_0's mape: 0.0682074
[3054]	valid_0's mape: 0.0682256
[3055]	valid_0's mape: 0.0682427
[3056]	valid_0's mape: 0.0682409
[3057]	valid_

[3276]	valid_0's mape: 0.0681697
[3277]	valid_0's mape: 0.0681784
[3278]	valid_0's mape: 0.0681625
[3279]	valid_0's mape: 0.0681505
[3280]	valid_0's mape: 0.0681603
[3281]	valid_0's mape: 0.0681895
[3282]	valid_0's mape: 0.0682003
[3283]	valid_0's mape: 0.0681833
[3284]	valid_0's mape: 0.0681649
[3285]	valid_0's mape: 0.0681924
[3286]	valid_0's mape: 0.0681768
[3287]	valid_0's mape: 0.0681602
[3288]	valid_0's mape: 0.068149
[3289]	valid_0's mape: 0.0681762
[3290]	valid_0's mape: 0.0681845
[3291]	valid_0's mape: 0.0681659
[3292]	valid_0's mape: 0.0681952
[3293]	valid_0's mape: 0.0681836
[3294]	valid_0's mape: 0.068201
[3295]	valid_0's mape: 0.068186
[3296]	valid_0's mape: 0.0681791
[3297]	valid_0's mape: 0.0681913
[3298]	valid_0's mape: 0.0681759
[3299]	valid_0's mape: 0.0681982
[3300]	valid_0's mape: 0.0682005
[3301]	valid_0's mape: 0.0682026
[3302]	valid_0's mape: 0.0681899
[3303]	valid_0's mape: 0.0681755
[3304]	valid_0's mape: 0.06818
[3305]	valid_0's mape: 0.0681842
[3306]	valid_0'

[3525]	valid_0's mape: 0.0681288
[3526]	valid_0's mape: 0.0681168
[3527]	valid_0's mape: 0.0681244
[3528]	valid_0's mape: 0.0681139
[3529]	valid_0's mape: 0.0681215
[3530]	valid_0's mape: 0.0681338
[3531]	valid_0's mape: 0.0681684
[3532]	valid_0's mape: 0.0681935
[3533]	valid_0's mape: 0.0681789
[3534]	valid_0's mape: 0.0681652
[3535]	valid_0's mape: 0.0681948
[3536]	valid_0's mape: 0.0681784
[3537]	valid_0's mape: 0.068175
[3538]	valid_0's mape: 0.0681841
[3539]	valid_0's mape: 0.0682023
[3540]	valid_0's mape: 0.0682043
[3541]	valid_0's mape: 0.0682282
[3542]	valid_0's mape: 0.0682628
[3543]	valid_0's mape: 0.0682858
[3544]	valid_0's mape: 0.0683058
[3545]	valid_0's mape: 0.0682855
[3546]	valid_0's mape: 0.068263
[3547]	valid_0's mape: 0.0682774
[3548]	valid_0's mape: 0.0683002
[3549]	valid_0's mape: 0.0682801
[3550]	valid_0's mape: 0.0682872
[3551]	valid_0's mape: 0.0682727
[3552]	valid_0's mape: 0.0682778
[3553]	valid_0's mape: 0.0682995
[3554]	valid_0's mape: 0.068295
[3555]	valid_

[3774]	valid_0's mape: 0.068036
[3775]	valid_0's mape: 0.0680616
[3776]	valid_0's mape: 0.0680473
[3777]	valid_0's mape: 0.06804
[3778]	valid_0's mape: 0.0680312
[3779]	valid_0's mape: 0.0680368
[3780]	valid_0's mape: 0.0680577
[3781]	valid_0's mape: 0.0680719
[3782]	valid_0's mape: 0.0680793
[3783]	valid_0's mape: 0.068095
[3784]	valid_0's mape: 0.0680806
[3785]	valid_0's mape: 0.0680898
[3786]	valid_0's mape: 0.0681119
[3787]	valid_0's mape: 0.0680991
[3788]	valid_0's mape: 0.0680835
[3789]	valid_0's mape: 0.0680683
[3790]	valid_0's mape: 0.0680583
[3791]	valid_0's mape: 0.0680655
[3792]	valid_0's mape: 0.0680754
[3793]	valid_0's mape: 0.0681021
[3794]	valid_0's mape: 0.0681094
[3795]	valid_0's mape: 0.0680971
[3796]	valid_0's mape: 0.0681135
[3797]	valid_0's mape: 0.0681292
[3798]	valid_0's mape: 0.0681135
[3799]	valid_0's mape: 0.0681413
[3800]	valid_0's mape: 0.0681532
[3801]	valid_0's mape: 0.0681747
[3802]	valid_0's mape: 0.068154
[3803]	valid_0's mape: 0.0681793
[3804]	valid_0'

5it [14:09, 170.09s/it]


In [6]:
#ハイパーパラメータ確認
Parameter_df

""


In [7]:
#各回でのMAPE検証値
MAPE_df

,MAPE
1,7.191017
1,6.723687
1,6.622930
1,6.932635
1,6.804854


In [8]:
MAPE_train_df

,MAPE
1,2.662512
1,2.680875
1,2.704113
1,2.704204
1,2.674498


In [9]:
compared_train.sort_values('APE',ascending=False)

,0,keiyaku_pr,APE
1469,4.691172e+06,3500000,0.340335
3722,1.351600e+07,10800000,0.251481
2885,2.930324e+07,24000000,0.220968
537,1.589137e+07,19900000,0.201439
1527,8.612864e+06,7300000,0.179844
3719,1.970623e+07,17000000,0.159190
3718,1.970623e+07,17000000,0.159190
3498,1.954090e+07,17000000,0.149465
1712,2.524942e+07,29500000,0.144087
4832,1.827815e+07,16000000,0.142385


In [10]:
compared.sort_values('APE',ascending=False)

,0,keiyaku_pr,APE
1109,1.174552e+07,5500000,1.135550
435,2.706696e+07,18500000,0.463079
926,1.087808e+07,7900000,0.376972
1097,2.705065e+07,19990000,0.353209
486,2.116742e+07,31600000,0.330145
963,7.886088e+06,6000000,0.314348
638,1.559145e+07,11990000,0.300371
26,2.633339e+07,20300000,0.297211
578,5.901443e+07,46300000,0.274610
1270,2.251504e+07,17800000,0.264890


In [11]:
MAPE_df.mean()

0
MAPE    6.855025
dtype: float64

In [12]:
importance_df = pd.DataFrame([model.feature_importance(importance_type='gain'),X.columns]).T
importance_df.rename(columns={0:'importance',1:'feature_name'},inplace =True)
importance_df.set_index('feature_name',inplace=True)

In [13]:
importance_df.sort_values('importance',ascending = False)

,importance
feature_name,
koji_tatemenseki,1.44874e+18
koji_totimenseki,7.94429e+17
rosenka_hb,7.69624e+17
kijun_tatemenseki,7.6042e+17
kijun_totimenseki,3.3119e+17
tt_mseki,3.14869e+17
toho_kyori1,2.14553e+17
eki_nm1,1.94583e+17
latitude,1.94161e+17


In [14]:
importance_df.query('importance >0').index.tolist()

['tc_mseki',
 'tt_mseki',
 'levelplan',
 'fukuin',
 'magutchi',
 'setsudo_hi',
 'setsudo_kj',
 'jigata',
 'hiatari',
 'garage',
 'kobetsu1',
 'jukyo',
 'chiseki_js_hb',
 'yoto1',
 'gesui',
 'gas',
 'usui',
 'tochiuri_su',
 'joken_su',
 'hy2f_date_su',
 'hy3f_date_su',
 'road1_hk',
 'road1_sb',
 'road2_hk',
 'road2_sb',
 'road3_sb',
 'road3_fi',
 'road4_sb',
 'kaoku_um',
 'kaoku_hb',
 'yheki_kotei',
 'yheki_umu',
 'yheki_yohi',
 'gk_sho_kyori',
 'gk_chu_kyori',
 'kborjs',
 'hw_status',
 'toshikuiki1',
 'kodochiku',
 'chikukeikaku',
 'keikakuroad',
 'kaihatsukyoka',
 't53kyoka',
 'hokakyoka',
 'bokachiiki',
 'minmenseki',
 'hokakisei1',
 'kinshijiko',
 'rosenka_hb',
 'koji_hb',
 'kijun_hb',
 'mseki_rd_hb',
 'mseki_dp_hb',
 'tt_mseki_min_hb',
 'tt_mseki_max_hb',
 'tc_mseki_avg_hb',
 'fi4m_yohi',
 'bus_yohi',
 'bus_hon',
 'sho_conv',
 'sho_super',
 'sho_shoten',
 'sho_market',
 'shu_jutaku',
 'shu_park',
 'shu_factory',
 'shu_bochi',
 'shu_kaido',
 'shu_line_ari',
 'shu_line_nashi',
 'shu_

In [15]:
importance_df.query('importance >0').shape

(164, 1)

In [16]:
importance_df.shape

(202, 1)

In [17]:
#必要な列だけ残す
X = train_input.drop(['id','pj_no','keiyaku_pr'],axis = 1)
X_test = test_input.drop(['id','pj_no'],axis = 1)
y = train_input[['keiyaku_pr']]

Parameter_df =pd.DataFrame()
MAPE_df = pd.DataFrame()
MAPE_train_df =pd.DataFrame()
Submit_df = test_input[['id']].copy(deep = True)
Submit_df['MAPE_sum'] = 0


for i in np.arange(5):


#     # 学習データと検証データに分割する
    X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size = 0.2,shuffle = True,random_state = i)
#     X_train, X_valid, y_train, y_valid = GroupShuffleSplit(X, y,groups=train_input['pj_no'].array,test_size = 0.2,shuffle = True,random_state = i+10)

    # 上記のパラメータでモデルを学習する

    #'''
#     model = LGBMRegressorCV(
#                             n_estimators = 100,
#                             learning_rate = 0.1,
#                             n_trials = 100,
#                             n_iter_no_change=10,
#                             random_state=0,
#                             timeout = 60)
#     #'''

#     #ハイパーパラメータチューニングしない場合はこっち
#     #model = lgb.LGBMRegressor(random_state=10)

#     #学習
#     model.fit(X_train, y_train,categorical_feature=important_categorical_columns)

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    # LightGBM parameters
    params = {
            'task' : 'train',
            'boosting_type' : 'dart',
            'objective' : 'regression',
            'metric' : {'mape'},
            'num_leaves' : 20,
            'learning_rate' :  0.05,
#             'n_estimators' : 3000,
            'feature_fraction' : 0.5,
            'bagging_fraction' : 0.8,
            'bagging_freq': 5,
            'verbose' : 0
#             'random_state':100
    }


    # train
    model = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval)
#                 early_stopping_rounds=1000)

    #選ばれたハイパーパラメータの記録
    #temp_parameter_df = pd.DataFrame(model.best_params_.items()).set_index(0).T
    #Parameter_df = Parameter_df.append(temp_parameter_df )

    # 検証データを予測する
    #y_pred = model.predict(X_valid)
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)

    #検証スコア算出
    y_pred_df = pd.DataFrame(y_pred)
    y_valid = y_valid.reset_index().drop('index',axis=1)
    compared = pd.concat([y_pred_df,y_valid ],axis = 1)
    compared['APE'] = abs((compared[0]-compared['keiyaku_pr'])/compared['keiyaku_pr'])
    
    #検証スコア算出(train)
    y_pred_train_df = pd.DataFrame(y_pred_train)
    y_train = y_train.reset_index().drop('index',axis=1)
    compared_train = pd.concat([y_pred_train_df,y_train ],axis = 1)
    compared_train['APE'] = abs((compared_train[0]-compared_train['keiyaku_pr'])/compared_train['keiyaku_pr'])

    #検証スコアを記録
    temp_dict = {'MAPE':compared['APE'].mean()*100}
    temp_df = pd.DataFrame(temp_dict.items()).set_index(0).T
    MAPE_df = MAPE_df.append(temp_df)

    #検証スコアを記録
    temp_dict_train = {'MAPE':compared_train['APE'].mean()*100}
    temp_train_df = pd.DataFrame(temp_dict_train.items()).set_index(0).T
    MAPE_train_df = MAPE_train_df.append(temp_train_df)
    
    #予測を実行
    #y_test_pred = model.predict(X_test)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_test_pred_df = pd.DataFrame(y_test_pred)

    #提出用のデータフレーム作成
    temp_submit = pd.concat([test_input[['id']],y_test_pred_df],axis=1)
    temp_submit.rename(columns={0:'MAPE_'+str(i)},inplace = True)
    Submit_df = pd.merge(Submit_df,temp_submit,on='id')

    Submit_df['MAPE_sum'] = Submit_df['MAPE_sum'] + Submit_df['MAPE_'+str(i)]
    i = i+1
    
Submit_df['MAPE_mean'] = Submit_df['MAPE_sum']/(i)
    
    

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's mape: 0.209856
[2]	valid_0's mape: 0.204096
[3]	valid_0's mape: 0.198019
[4]	valid_0's mape: 0.19277
[5]	valid_0's mape: 0.187867
[6]	valid_0's mape: 0.183139
[7]	valid_0's mape: 0.178322
[8]	valid_0's mape: 0.180072
[9]	valid_0's mape: 0.175029
[10]	valid_0's mape: 0.170826
[11]	valid_0's mape: 0.16707
[12]	valid_0's mape: 0.168378
[13]	valid_0's mape: 0.164681
[14]	valid_0's mape: 0.161158
[15]	valid_0's mape: 0.157545
[16]	valid_0's mape: 0.15422
[17]	valid_0's mape: 0.151084
[18]	valid_0's mape: 0.148021
[19]	valid_0's mape: 0.1454
[20]	valid_0's mape: 0.142658
[21]	valid_0's mape: 0.143496
[22]	valid_0's mape: 0.14106
[23]	valid_0's mape: 0.13874
[24]	valid_0's mape: 0.136443
[25]	valid_0's mape: 0.134522
[26]	valid_0's mape: 0.132582
[27]	valid_0's mape: 0.130827
[28]	valid_0's mape: 0.131645
[29]	valid_0's mape: 0.129822
[30]	valid_0's mape: 0.128069
[31]	valid_0's mape: 0.128419
[32]	valid_0's mape: 0.12681
[33]	valid_0's mape: 0.125356
[34]	valid_0's mape: 0.1238

[269]	valid_0's mape: 0.0999819
[270]	valid_0's mape: 0.0985735
[271]	valid_0's mape: 0.100109
[272]	valid_0's mape: 0.0985773
[273]	valid_0's mape: 0.0972536
[274]	valid_0's mape: 0.0984427
[275]	valid_0's mape: 0.0971404
[276]	valid_0's mape: 0.0986317
[277]	valid_0's mape: 0.0973016
[278]	valid_0's mape: 0.0984604
[279]	valid_0's mape: 0.0972637
[280]	valid_0's mape: 0.0985633
[281]	valid_0's mape: 0.103955
[282]	valid_0's mape: 0.102075
[283]	valid_0's mape: 0.103588
[284]	valid_0's mape: 0.101823
[285]	valid_0's mape: 0.100178
[286]	valid_0's mape: 0.0986936
[287]	valid_0's mape: 0.0973681
[288]	valid_0's mape: 0.0988425
[289]	valid_0's mape: 0.0975229
[290]	valid_0's mape: 0.096314
[291]	valid_0's mape: 0.0950944
[292]	valid_0's mape: 0.0964128
[293]	valid_0's mape: 0.0975922
[294]	valid_0's mape: 0.0962624
[295]	valid_0's mape: 0.0974435
[296]	valid_0's mape: 0.0985841
[297]	valid_0's mape: 0.0972608
[298]	valid_0's mape: 0.0984209
[299]	valid_0's mape: 0.0971079
[300]	valid_0's

[526]	valid_0's mape: 0.0818065
[527]	valid_0's mape: 0.0830703
[528]	valid_0's mape: 0.0825983
[529]	valid_0's mape: 0.082131
[530]	valid_0's mape: 0.0817792
[531]	valid_0's mape: 0.0821904
[532]	valid_0's mape: 0.0817661
[533]	valid_0's mape: 0.0821523
[534]	valid_0's mape: 0.0816916
[535]	valid_0's mape: 0.0812789
[536]	valid_0's mape: 0.0816206
[537]	valid_0's mape: 0.0812288
[538]	valid_0's mape: 0.080894
[539]	valid_0's mape: 0.0812444
[540]	valid_0's mape: 0.0816406
[541]	valid_0's mape: 0.0821086
[542]	valid_0's mape: 0.0816771
[543]	valid_0's mape: 0.082055
[544]	valid_0's mape: 0.0824826
[545]	valid_0's mape: 0.0820749
[546]	valid_0's mape: 0.0816456
[547]	valid_0's mape: 0.0812659
[548]	valid_0's mape: 0.0809059
[549]	valid_0's mape: 0.0812623
[550]	valid_0's mape: 0.0809426
[551]	valid_0's mape: 0.0806696
[552]	valid_0's mape: 0.0803962
[553]	valid_0's mape: 0.0807087
[554]	valid_0's mape: 0.081053
[555]	valid_0's mape: 0.080787
[556]	valid_0's mape: 0.0811662
[557]	valid_0

[782]	valid_0's mape: 0.0788424
[783]	valid_0's mape: 0.0791297
[784]	valid_0's mape: 0.0794196
[785]	valid_0's mape: 0.0790777
[786]	valid_0's mape: 0.0793494
[787]	valid_0's mape: 0.0796297
[788]	valid_0's mape: 0.0799149
[789]	valid_0's mape: 0.0794812
[790]	valid_0's mape: 0.0791457
[791]	valid_0's mape: 0.0787908
[792]	valid_0's mape: 0.0790235
[793]	valid_0's mape: 0.0786937
[794]	valid_0's mape: 0.0789596
[795]	valid_0's mape: 0.0786578
[796]	valid_0's mape: 0.0789035
[797]	valid_0's mape: 0.0791879
[798]	valid_0's mape: 0.0794462
[799]	valid_0's mape: 0.0790577
[800]	valid_0's mape: 0.0793321
[801]	valid_0's mape: 0.0790032
[802]	valid_0's mape: 0.0792697
[803]	valid_0's mape: 0.0795061
[804]	valid_0's mape: 0.0798074
[805]	valid_0's mape: 0.0800489
[806]	valid_0's mape: 0.0803423
[807]	valid_0's mape: 0.0799185
[808]	valid_0's mape: 0.0795369
[809]	valid_0's mape: 0.0798041
[810]	valid_0's mape: 0.0793891
[811]	valid_0's mape: 0.079044
[812]	valid_0's mape: 0.079299
[813]	vali

...

[2865]	valid_0's mape: 0.0694898
[2866]	valid_0's mape: 0.0694872
[2867]	valid_0's mape: 0.0694845
[2868]	valid_0's mape: 0.0694826
[2869]	valid_0's mape: 0.0694852
[2870]	valid_0's mape: 0.0694872
[2871]	valid_0's mape: 0.069483
[2872]	valid_0's mape: 0.0694726
[2873]	valid_0's mape: 0.0694765
[2874]	valid_0's mape: 0.0694813
[2875]	valid_0's mape: 0.0694794
[2876]	valid_0's mape: 0.0694788
[2877]	valid_0's mape: 0.0694859
[2878]	valid_0's mape: 0.0694796
[2879]	valid_0's mape: 0.0694951
[2880]	valid_0's mape: 0.0694979
[2881]	valid_0's mape: 0.0695088
[2882]	valid_0's mape: 0.0695218
[2883]	valid_0's mape: 0.0695272
[2884]	valid_0's mape: 0.0695195
[2885]	valid_0's mape: 0.0695414
[2886]	valid_0's mape: 0.069493
[2887]	valid_0's mape: 0.0694973
[2888]	valid_0's mape: 0.0694991
[2889]	valid_0's mape: 0.0695023
[2890]	valid_0's mape: 0.0694977
[2891]	valid_0's mape: 0.069498
[2892]	valid_0's mape: 0.0694997
[2893]	valid_0's mape: 0.0694892
[2894]	valid_0's mape: 0.0694952
[2895]	valid_

[3114]	valid_0's mape: 0.069538
[3115]	valid_0's mape: 0.0695599
[3116]	valid_0's mape: 0.0695528
[3117]	valid_0's mape: 0.0695483
[3118]	valid_0's mape: 0.0695469
[3119]	valid_0's mape: 0.0695431
[3120]	valid_0's mape: 0.0695614
[3121]	valid_0's mape: 0.0695538
[3122]	valid_0's mape: 0.0695682
[3123]	valid_0's mape: 0.0695753
[3124]	valid_0's mape: 0.0695662
[3125]	valid_0's mape: 0.0695603
[3126]	valid_0's mape: 0.0695496
[3127]	valid_0's mape: 0.0695411
[3128]	valid_0's mape: 0.0695359
[3129]	valid_0's mape: 0.0695322
[3130]	valid_0's mape: 0.0695273
[3131]	valid_0's mape: 0.0695361
[3132]	valid_0's mape: 0.0695309
[3133]	valid_0's mape: 0.0695372
[3134]	valid_0's mape: 0.0695275
[3135]	valid_0's mape: 0.0695214
[3136]	valid_0's mape: 0.0695391
[3137]	valid_0's mape: 0.0695457
[3138]	valid_0's mape: 0.069559
[3139]	valid_0's mape: 0.0695771
[3140]	valid_0's mape: 0.0695677
[3141]	valid_0's mape: 0.0695554
[3142]	valid_0's mape: 0.0695747
[3143]	valid_0's mape: 0.0695629
[3144]	valid

[3363]	valid_0's mape: 0.0695154
[3364]	valid_0's mape: 0.0695215
[3365]	valid_0's mape: 0.0694972
[3366]	valid_0's mape: 0.0695075
[3367]	valid_0's mape: 0.0694934
[3368]	valid_0's mape: 0.0694753
[3369]	valid_0's mape: 0.0694887
[3370]	valid_0's mape: 0.0695074
[3371]	valid_0's mape: 0.0695287
[3372]	valid_0's mape: 0.0695373
[3373]	valid_0's mape: 0.0695522
[3374]	valid_0's mape: 0.0695634
[3375]	valid_0's mape: 0.0695478
[3376]	valid_0's mape: 0.0695522
[3377]	valid_0's mape: 0.069534
[3378]	valid_0's mape: 0.0695165
[3379]	valid_0's mape: 0.069521
[3380]	valid_0's mape: 0.0695135
[3381]	valid_0's mape: 0.0695213
[3382]	valid_0's mape: 0.069522
[3383]	valid_0's mape: 0.0695016
[3384]	valid_0's mape: 0.069486
[3385]	valid_0's mape: 0.0694684
[3386]	valid_0's mape: 0.0694679
[3387]	valid_0's mape: 0.0694596
[3388]	valid_0's mape: 0.069468
[3389]	valid_0's mape: 0.0694573
[3390]	valid_0's mape: 0.0694417
[3391]	valid_0's mape: 0.0694277
[3392]	valid_0's mape: 0.0694169
[3393]	valid_0'

[3612]	valid_0's mape: 0.0692467
[3613]	valid_0's mape: 0.069248
[3614]	valid_0's mape: 0.0692502
[3615]	valid_0's mape: 0.0692535
[3616]	valid_0's mape: 0.069251
[3617]	valid_0's mape: 0.0692505
[3618]	valid_0's mape: 0.0692582
[3619]	valid_0's mape: 0.0692601
[3620]	valid_0's mape: 0.0692585
[3621]	valid_0's mape: 0.0692604
[3622]	valid_0's mape: 0.069257
[3623]	valid_0's mape: 0.0692524
[3624]	valid_0's mape: 0.0692387
[3625]	valid_0's mape: 0.0692295
[3626]	valid_0's mape: 0.0692472
[3627]	valid_0's mape: 0.0692526
[3628]	valid_0's mape: 0.0692564
[3629]	valid_0's mape: 0.06927
[3630]	valid_0's mape: 0.0692626
[3631]	valid_0's mape: 0.0692863
[3632]	valid_0's mape: 0.069277
[3633]	valid_0's mape: 0.0692851
[3634]	valid_0's mape: 0.0692766
[3635]	valid_0's mape: 0.0692688
[3636]	valid_0's mape: 0.0692608
[3637]	valid_0's mape: 0.0692765
[3638]	valid_0's mape: 0.0692696
[3639]	valid_0's mape: 0.0692667
[3640]	valid_0's mape: 0.0692608
[3641]	valid_0's mape: 0.069258
[3642]	valid_0's 

[3861]	valid_0's mape: 0.0693788
[3862]	valid_0's mape: 0.0693717
[3863]	valid_0's mape: 0.0693608
[3864]	valid_0's mape: 0.0693631
[3865]	valid_0's mape: 0.0693549
[3866]	valid_0's mape: 0.0693474
[3867]	valid_0's mape: 0.0693375
[3868]	valid_0's mape: 0.0693567
[3869]	valid_0's mape: 0.0693496
[3870]	valid_0's mape: 0.0693631
[3871]	valid_0's mape: 0.0693581
[3872]	valid_0's mape: 0.0693745
[3873]	valid_0's mape: 0.0693653
[3874]	valid_0's mape: 0.0693735
[3875]	valid_0's mape: 0.0693627
[3876]	valid_0's mape: 0.0693641
[3877]	valid_0's mape: 0.0693598
[3878]	valid_0's mape: 0.0693535
[3879]	valid_0's mape: 0.069367
[3880]	valid_0's mape: 0.0693589
[3881]	valid_0's mape: 0.0693506
[3882]	valid_0's mape: 0.0693593
[3883]	valid_0's mape: 0.0693718
[3884]	valid_0's mape: 0.0693616
[3885]	valid_0's mape: 0.0693644
[3886]	valid_0's mape: 0.0693626
[3887]	valid_0's mape: 0.0693594
[3888]	valid_0's mape: 0.0693471
[3889]	valid_0's mape: 0.0693741
[3890]	valid_0's mape: 0.0693639
[3891]	vali

In [18]:
Submit_df

,id,MAPE_sum,MAPE_0,MAPE_1,MAPE_2,MAPE_3,MAPE_4,MAPE_mean
0,test_0000,9.636704e+07,1.840001e+07,1.951161e+07,1.938729e+07,1.990425e+07,1.916388e+07,1.927341e+07
1,test_0001,1.323366e+08,2.645611e+07,2.664667e+07,2.614526e+07,2.650280e+07,2.658575e+07,2.646732e+07
2,test_0002,1.260800e+08,2.499154e+07,2.565333e+07,2.520184e+07,2.500778e+07,2.522549e+07,2.521600e+07
3,test_0003,1.390169e+08,2.765984e+07,2.813456e+07,2.794023e+07,2.773662e+07,2.754564e+07,2.780338e+07
4,test_0004,1.338270e+08,2.658938e+07,2.679629e+07,2.671053e+07,2.647326e+07,2.725750e+07,2.676539e+07
5,test_0005,1.235053e+08,2.456696e+07,2.523352e+07,2.458288e+07,2.439827e+07,2.472367e+07,2.470106e+07
6,test_0006,1.335903e+08,2.644130e+07,2.694362e+07,2.692434e+07,2.643781e+07,2.684322e+07,2.671806e+07
7,test_0007,1.266202e+08,2.539875e+07,2.570831e+07,2.550700e+07,2.459823e+07,2.540790e+07,2.532404e+07
8,test_0008,1.317529e+08,2.637899e+07,2.658873e+07,2.600243e+07,2.633420e+07,2.644857e+07,2.635059e+07
9,test_0009,1.478532e+08,2.901464e+07,2.974942e+07,2.983402e+07,2.958067e+07,2.967446e+07,2.957064e+07


In [19]:
#Submit_df[['id','MAPE_mean']].to_csv('submit_0812_1.tsv',header=False,index=False,sep ='\t')

In [20]:
test_predict = Submit_df[['MAPE_mean']].rename(columns={'MAPE_mean':'keiyaku_pr'})

In [21]:
test_predict

,keiyaku_pr
0,1.927341e+07
1,2.646732e+07
2,2.521600e+07
3,2.780338e+07
4,2.676539e+07
5,2.470106e+07
6,2.671806e+07
7,2.532404e+07
8,2.635059e+07
9,2.957064e+07


In [22]:
X = train_input.drop(['id','pj_no','keiyaku_pr'],axis = 1).append(test_input.drop(['id','pj_no'],axis = 1))

In [26]:
X.head()

,tc_mseki,tt_mseki,levelplan,fukuin,road_st,magutchi,setsudo_hi,setsudo_kj,jigata,hiatari,garage,kobetsu1,kobetsu2,kobetsu3,kobetsu4,jukyo,chiseki_js_hb,yoto1,yoto2,josui,gesui,gas,usui,tochiuri_su,joken_su,hy2f_date_su,hy3f_date_su,road1_hk,road1_sb,road2_hk,road2_sb,road3_hk,road3_sb,road3_fi,road4_hk,road4_sb,kaoku_um,kaoku_hb,yheki_kotei,yheki_umu,yheki_yohi,gk_sho_kyori,gk_chu_kyori,kborjs,hw_status,toshikuiki1,toshikuiki2,kodochiku,chikukeikaku,keikakuroad,kaihatsukyoka,t53kyoka,hokakyoka,bokachiiki,minmenseki,hokakisei1,hokakisei2,hokakisei3,hokakisei4,kinshijiko,rosenka_hb,koji_hb,kijun_hb,mseki_rd_hb,mseki_dp_hb,tt_mseki_min_hb,tt_mseki_max_hb,tc_mseki_avg_hb,fi4m_yohi,fi3m_yohi,bus_yohi,bus_hon,sho_conv,sho_super,sho_shoten,sho_market,shu_jutaku,shu_park,shu_shop,shu_factory,shu_hvline,shu_tower,shu_bochi,shu_sogi,shu_zoki,shu_kokyo,shu_highway,shu_kaido,shu_line_ari,shu_line_nashi,shu_soon,gk_yoc_tm,gk_sho_tm,gk_chu_tm,rs_e_kdate2,rs_e_kdate3,rs_e_parking,rs_e_zoki,rs_e_m_ari,rs_e_m_nashi,...,rs_w_kdate3,rs_w_parking,rs_w_zoki,rs_w_m_ari,rs_w_m_nashi,rs_w_tahata,rs_s_kdate2,rs_s_kdate3,rs_s_parking,rs_s_zoki,rs_s_m_ari,rs_s_m_nashi,rs_s_tahata,rs_n_kdate2,rs_n_kdate3,rs_n_parking,rs_n_zoki,rs_n_m_ari,rs_n_m_nashi,rs_n_tahata,rosen_nm1,eki_nm1,bas_toho1,eki_kyori1,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2,city1,city2,city3,longitude,latitude,階数,部屋数,posi_nega,bus_kyori1,toho_kyori1,car_kyori1,bus_kyori2,toho_kyori2,car_kyori2,kaimono,kijun_tatemenseki,koji_tatemenseki,kijun_totimenseki,koji_totimenseki,menseki_hi,count_city1,count_city2,count_city3,count_階数,count_部屋数,count_jukyo,count_yoto1,count_yoto2,count_josui,count_gesui,count_gas,count_usui,count_road1_hk,count_road1_sb,count_road2_hk,count_road2_sb,count_road3_hk,count_road3_sb,count_road4_hk,count_road4_sb,count_hw_status,count_toshikuiki1,count_toshikuiki2,count_kodochiku,count_bokachiiki,count_hokakisei1,count_hokakisei2,count_hokakisei3,count_hokakisei4,count_kinshijiko,count_rosen_nm1,count_eki_nm1,count_bas_toho1,count_bastei_nm1,count_rosen_nm2,count_eki_nm2,count_bas_toho2,count_bastei_nm2,count_levelplan,count_road_st,count_setsudo_hi,count_setsudo_kj,count_jigata,count_hiatari,count_kobetsu1,count_kobetsu2,count_kobetsu3,count_kobetsu4
0,109.26,104.43,2F/4LDK,4.0,問題なし,9.9,東,良い,整形地,普通,1,床暖房付,NaN,NaN,NaN,埼玉県朝霞市泉水,109.26,工業地域,NaN,公営,公共下水,個別プロパン,宅内処理,0.0,0.0,1.0,0.0,北,私道,東,私道,NaN,NaN,0.0,NaN,NaN,False,0.0,0.6,False,False,800,1000,False,更地,市街化区域,NaN,第2種,False,False,False,False,False,防火指定無,0.0,NaN,NaN,NaN,NaN,0,135000.0,188000,197000.0,NaN,0.00,104.43,104.43,109.26,False,False,False,NaN,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,19.0,10,13,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,東武東上線,朝霞台,徒歩,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,埼玉県朝霞,埼玉県朝霞,埼玉県朝霞,35.797086,139.593733,2F,4LDK,1.0,NaN,17.0,NaN,NaN,NaN,NaN,False,20572710.0,19632840.0,21524220.0,20540880.0,1.000000,113,113,113,9579.0,8639.0,1,162,NaN,10651,9394,2859,9977,2085,898.0,896.0,570.0,NaN,NaN,NaN,NaN,6182,9846,NaN,365,5982,NaN,NaN,NaN,NaN,10520,1397,34,8353,NaN,NaN,NaN,NaN,NaN,8376.0,10047.0,1736.0,5693.0,8021.0,4352.0,112.0,NaN,NaN,NaN
1,136.11,105.16,2F/4LDK,4.2,問題なし,9.3,北西,普通,不整形地,普通,1,NaN,NaN,NaN,NaN,埼玉県川越市大字的場,136.15,第一種低層住居専用地域,NaN,公営,公共下水,個別プロパン,宅内処理,0.0,0.0,1.0,0.0,北西,公道,NaN,NaN,NaN,NaN,0.0,NaN,NaN,False,0.0,0.0,False,False,1680,1840,False,更地,市街化区域,NaN,指定無,False,False,False,False,False,22条区域,NaN,NaN,農地法,NaN,NaN,0,68000.0,84900,48600.0,NaN,0.00,109.10,109.10,136.15,False,False,False,NaN,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,15.0,21,23,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,JR川越線,的場,徒歩,13,Na

In [23]:
#必要な列だけ残す
X = train_input.drop(['id','pj_no','keiyaku_pr'],axis = 1).append(test_input.drop(['id','pj_no'],axis = 1))
X_test = test_input.drop(['id','pj_no'],axis = 1)
y = train_input[['keiyaku_pr']].append(test_predict)

#LightGBMでのカテゴリ扱いをする変数を指定
for column in category_columns:
    X[column] = X[column].astype('category')


Parameter_df =pd.DataFrame()
MAPE_df = pd.DataFrame()
MAPE_train_df =pd.DataFrame()
Submit_df = test_input[['id']].copy(deep = True)
Submit_df['MAPE_sum'] = 0


for i in np.arange(5):


#     # 学習データと検証データに分割する
    X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size = 0.2,shuffle = True,random_state = i)
#     X_train, X_valid, y_train, y_valid = GroupShuffleSplit(X, y,groups=train_input['pj_no'].array,test_size = 0.2,shuffle = True,random_state = i+10)

    # 上記のパラメータでモデルを学習する

    #'''
#     model = LGBMRegressorCV(
#                             n_estimators = 100,
#                             learning_rate = 0.1,
#                             n_trials = 100,
#                             n_iter_no_change=10,
#                             random_state=0,
#                             timeout = 60)
#     #'''

#     #ハイパーパラメータチューニングしない場合はこっち
#     #model = lgb.LGBMRegressor(random_state=10)

#     #学習
#     model.fit(X_train, y_train,categorical_feature=important_categorical_columns)

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    # LightGBM parameters
    params = {
            'task' : 'train',
            'boosting_type' : 'dart',
            'objective' : 'regression',
            'metric' : {'mape'},
            'num_leaves' : 20,
            'learning_rate' :  0.05,
#             'n_estimators' : 3000,
            'feature_fraction' : 0.5,
            'bagging_fraction' : 0.8,
            'bagging_freq': 5,
            'verbose' : 0
#             'random_state':100
    }


    # train
    model = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval)
#                 early_stopping_rounds=1000)

    #選ばれたハイパーパラメータの記録
    #temp_parameter_df = pd.DataFrame(model.best_params_.items()).set_index(0).T
    #Parameter_df = Parameter_df.append(temp_parameter_df )

    # 検証データを予測する
    #y_pred = model.predict(X_valid)
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)

    #検証スコア算出
    y_pred_df = pd.DataFrame(y_pred)
    y_valid = y_valid.reset_index().drop('index',axis=1)
    compared = pd.concat([y_pred_df,y_valid ],axis = 1)
    compared['APE'] = abs((compared[0]-compared['keiyaku_pr'])/compared['keiyaku_pr'])
    
    #検証スコア算出(train)
    y_pred_train_df = pd.DataFrame(y_pred_train)
    y_train = y_train.reset_index().drop('index',axis=1)
    compared_train = pd.concat([y_pred_train_df,y_train ],axis = 1)
    compared_train['APE'] = abs((compared_train[0]-compared_train['keiyaku_pr'])/compared_train['keiyaku_pr'])

    #検証スコアを記録
    temp_dict = {'MAPE':compared['APE'].mean()*100}
    temp_df = pd.DataFrame(temp_dict.items()).set_index(0).T
    MAPE_df = MAPE_df.append(temp_df)

    #検証スコアを記録
    temp_dict_train = {'MAPE':compared_train['APE'].mean()*100}
    temp_train_df = pd.DataFrame(temp_dict_train.items()).set_index(0).T
    MAPE_train_df = MAPE_train_df.append(temp_train_df)
    
    #予測を実行
    #y_test_pred = model.predict(X_test)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_test_pred_df = pd.DataFrame(y_test_pred)

    #提出用のデータフレーム作成
    temp_submit = pd.concat([test_input[['id']],y_test_pred_df],axis=1)
    temp_submit.rename(columns={0:'MAPE_'+str(i)},inplace = True)
    Submit_df = pd.merge(Submit_df,temp_submit,on='id')

    Submit_df['MAPE_sum'] = Submit_df['MAPE_sum'] + Submit_df['MAPE_'+str(i)]
    i = i+1
    
Submit_df['MAPE_mean'] = Submit_df['MAPE_sum']/(i)
    
    

[1]	valid_0's mape: 0.190583
[2]	valid_0's mape: 0.184629
[3]	valid_0's mape: 0.178871
[4]	valid_0's mape: 0.173666
[5]	valid_0's mape: 0.168183
[6]	valid_0's mape: 0.163426
[7]	valid_0's mape: 0.158692
[8]	valid_0's mape: 0.160759
[9]	valid_0's mape: 0.156597
[10]	valid_0's mape: 0.15239
[11]	valid_0's mape: 0.148271
[12]	valid_0's mape: 0.149932
[13]	valid_0's mape: 0.145875
[14]	valid_0's mape: 0.14231
[15]	valid_0's mape: 0.138729
[16]	valid_0's mape: 0.135334
[17]	valid_0's mape: 0.132163
[18]	valid_0's mape: 0.129405
[19]	valid_0's mape: 0.12674
[20]	valid_0's mape: 0.124077
[21]	valid_0's mape: 0.124962
[22]	valid_0's mape: 0.122362
[23]	valid_0's mape: 0.11991
[24]	valid_0's mape: 0.117542
[25]	valid_0's mape: 0.115327
[26]	valid_0's mape: 0.113163
[27]	valid_0's mape: 0.111159
[28]	valid_0's mape: 0.112123
[29]	valid_0's mape: 0.110106
[30]	valid_0's mape: 0.108321
[31]	valid_0's mape: 0.10871
[32]	valid_0's mape: 0.106739
[33]	valid_0's mape: 0.104962
[34]	valid_0's mape: 0.1

[268]	valid_0's mape: 0.0808676
[269]	valid_0's mape: 0.0829322
[270]	valid_0's mape: 0.0810084
[271]	valid_0's mape: 0.0831698
[272]	valid_0's mape: 0.0812735
[273]	valid_0's mape: 0.0794696
[274]	valid_0's mape: 0.0811552
[275]	valid_0's mape: 0.0793625
[276]	valid_0's mape: 0.081506
[277]	valid_0's mape: 0.0796593
[278]	valid_0's mape: 0.0812966
[279]	valid_0's mape: 0.0794447
[280]	valid_0's mape: 0.0813023
[281]	valid_0's mape: 0.0890559
[282]	valid_0's mape: 0.0864905
[283]	valid_0's mape: 0.0885323
[284]	valid_0's mape: 0.0860924
[285]	valid_0's mape: 0.0838498
[286]	valid_0's mape: 0.0817854
[287]	valid_0's mape: 0.0799387
[288]	valid_0's mape: 0.0820525
[289]	valid_0's mape: 0.0801039
[290]	valid_0's mape: 0.0783689
[291]	valid_0's mape: 0.0768281
[292]	valid_0's mape: 0.0787798
[293]	valid_0's mape: 0.0804744
[294]	valid_0's mape: 0.0787001
[295]	valid_0's mape: 0.0803862
[296]	valid_0's mape: 0.0819957
[297]	valid_0's mape: 0.0800623
[298]	valid_0's mape: 0.0817053
[299]	val

[524]	valid_0's mape: 0.060085
[525]	valid_0's mape: 0.0608434
[526]	valid_0's mape: 0.0601806
[527]	valid_0's mape: 0.0626948
[528]	valid_0's mape: 0.0618704
[529]	valid_0's mape: 0.0610808
[530]	valid_0's mape: 0.0603269
[531]	valid_0's mape: 0.0610945
[532]	valid_0's mape: 0.0603658
[533]	valid_0's mape: 0.0610799
[534]	valid_0's mape: 0.0603319
[535]	valid_0's mape: 0.0596599
[536]	valid_0's mape: 0.0603044
[537]	valid_0's mape: 0.0596393
[538]	valid_0's mape: 0.0590329
[539]	valid_0's mape: 0.0597003
[540]	valid_0's mape: 0.0604152
[541]	valid_0's mape: 0.0612743
[542]	valid_0's mape: 0.0605025
[543]	valid_0's mape: 0.0611877
[544]	valid_0's mape: 0.0619659
[545]	valid_0's mape: 0.0611603
[546]	valid_0's mape: 0.0604212
[547]	valid_0's mape: 0.0597518
[548]	valid_0's mape: 0.0591343
[549]	valid_0's mape: 0.0598001
[550]	valid_0's mape: 0.0591993
[551]	valid_0's mape: 0.0586104
[552]	valid_0's mape: 0.0581022
[553]	valid_0's mape: 0.0587033
[554]	valid_0's mape: 0.0593522
[555]	val

[781]	valid_0's mape: 0.0577659
[782]	valid_0's mape: 0.0571378
[783]	valid_0's mape: 0.0577249
[784]	valid_0's mape: 0.0583118
[785]	valid_0's mape: 0.057629
[786]	valid_0's mape: 0.0581744
[787]	valid_0's mape: 0.0587367
[788]	valid_0's mape: 0.0592754
[789]	valid_0's mape: 0.0585011
[790]	valid_0's mape: 0.0577998
[791]	valid_0's mape: 0.0571776
[792]	valid_0's mape: 0.0576438
[793]	valid_0's mape: 0.0570209
[794]	valid_0's mape: 0.0575526
[795]	valid_0's mape: 0.0569326
[796]	valid_0's mape: 0.0574214
[797]	valid_0's mape: 0.0579856
[798]	valid_0's mape: 0.0584901
[799]	valid_0's mape: 0.0578051
[800]	valid_0's mape: 0.0583455
[801]	valid_0's mape: 0.0576667
[802]	valid_0's mape: 0.0581928
[803]	valid_0's mape: 0.0586613
[804]	valid_0's mape: 0.0592305
[805]	valid_0's mape: 0.059683
[806]	valid_0's mape: 0.0602427
[807]	valid_0's mape: 0.0593935
[808]	valid_0's mape: 0.0586251
[809]	valid_0's mape: 0.0591318
[810]	valid_0's mape: 0.0583706
[811]	valid_0's mape: 0.0576979
[812]	vali

[1036]	valid_0's mape: 0.0519972
[1037]	valid_0's mape: 0.0517184
[1038]	valid_0's mape: 0.0519954
[1039]	valid_0's mape: 0.0523325
[1040]	valid_0's mape: 0.052664
[1041]	valid_0's mape: 0.0523172
[1042]	valid_0's mape: 0.0536452
[1043]	valid_0's mape: 0.0531917
[1044]	valid_0's mape: 0.0528078
[1045]	valid_0's mape: 0.0524655
[1046]	valid_0's mape: 0.0527518
[1047]	valid_0's mape: 0.0531247
[1048]	valid_0's mape: 0.0527141
[1049]	valid_0's mape: 0.0530471
[1050]	valid_0's mape: 0.0526528
[1051]	valid_0's mape: 0.0530018
[1052]	valid_0's mape: 0.0533087
[1053]	valid_0's mape: 0.0528644
[1054]	valid_0's mape: 0.0524988
[1055]	valid_0's mape: 0.0528599
[1056]	valid_0's mape: 0.0524761
[1057]	valid_0's mape: 0.0528619
[1058]	valid_0's mape: 0.053265
[1059]	valid_0's mape: 0.0535961
[1060]	valid_0's mape: 0.0539736
[1061]	valid_0's mape: 0.0543707
[1062]	valid_0's mape: 0.0547672
[1063]	valid_0's mape: 0.0541926
[1064]	valid_0's mape: 0.0545556
[1065]	valid_0's mape: 0.0549608
[1066]	valid

...

[2855]	valid_0's mape: 0.0440772
[2856]	valid_0's mape: 0.0441148
[2857]	valid_0's mape: 0.0444151
[2858]	valid_0's mape: 0.0443092
[2859]	valid_0's mape: 0.0443809
[2860]	valid_0's mape: 0.044453
[2861]	valid_0's mape: 0.0445409
[2862]	valid_0's mape: 0.0444281
[2863]	valid_0's mape: 0.0443347
[2864]	valid_0's mape: 0.0442527
[2865]	valid_0's mape: 0.0441596
[2866]	valid_0's mape: 0.0440926
[2867]	valid_0's mape: 0.044033
[2868]	valid_0's mape: 0.0440814
[2869]	valid_0's mape: 0.0441312
[2870]	valid_0's mape: 0.0441819
[2871]	valid_0's mape: 0.0441136
[2872]	valid_0's mape: 0.0440522
[2873]	valid_0's mape: 0.0439922
[2874]	valid_0's mape: 0.0439429
[2875]	valid_0's mape: 0.0439972
[2876]	valid_0's mape: 0.0440332
[2877]	valid_0's mape: 0.043987
[2878]	valid_0's mape: 0.0440387
[2879]	valid_0's mape: 0.043982
[2880]	valid_0's mape: 0.0439335
[2881]	valid_0's mape: 0.0438958
[2882]	valid_0's mape: 0.0438567
[2883]	valid_0's mape: 0.0438267
[2884]	valid_0's mape: 0.0438558
[2885]	valid_0

[3104]	valid_0's mape: 0.0436082
[3105]	valid_0's mape: 0.0436071
[3106]	valid_0's mape: 0.0435861
[3107]	valid_0's mape: 0.0436023
[3108]	valid_0's mape: 0.0435864
[3109]	valid_0's mape: 0.0436016
[3110]	valid_0's mape: 0.0436209
[3111]	valid_0's mape: 0.043641
[3112]	valid_0's mape: 0.0436647
[3113]	valid_0's mape: 0.0437043
[3114]	valid_0's mape: 0.0436667
[3115]	valid_0's mape: 0.0436405
[3116]	valid_0's mape: 0.0436685
[3117]	valid_0's mape: 0.0436934
[3118]	valid_0's mape: 0.0436617
[3119]	valid_0's mape: 0.043689
[3120]	valid_0's mape: 0.043663
[3121]	valid_0's mape: 0.0436907
[3122]	valid_0's mape: 0.0436565
[3123]	valid_0's mape: 0.0436339
[3124]	valid_0's mape: 0.0436617
[3125]	valid_0's mape: 0.0436277
[3126]	valid_0's mape: 0.0436547
[3127]	valid_0's mape: 0.0436988
[3128]	valid_0's mape: 0.0437466
[3129]	valid_0's mape: 0.0437817
[3130]	valid_0's mape: 0.0437384
[3131]	valid_0's mape: 0.0437016
[3132]	valid_0's mape: 0.0437326
[3133]	valid_0's mape: 0.0436908
[3134]	valid_

[3353]	valid_0's mape: 0.043349
[3354]	valid_0's mape: 0.0433474
[3355]	valid_0's mape: 0.0433408
[3356]	valid_0's mape: 0.0433314
[3357]	valid_0's mape: 0.0433268
[3358]	valid_0's mape: 0.0433275
[3359]	valid_0's mape: 0.0433311
[3360]	valid_0's mape: 0.043336
[3361]	valid_0's mape: 0.0433245
[3362]	valid_0's mape: 0.0433108
[3363]	valid_0's mape: 0.0433014
[3364]	valid_0's mape: 0.0432937
[3365]	valid_0's mape: 0.043298
[3366]	valid_0's mape: 0.0432861
[3367]	valid_0's mape: 0.0432903
[3368]	valid_0's mape: 0.0432971
[3369]	valid_0's mape: 0.0432906
[3370]	valid_0's mape: 0.0432948
[3371]	valid_0's mape: 0.043297
[3372]	valid_0's mape: 0.0432973
[3373]	valid_0's mape: 0.0432994
[3374]	valid_0's mape: 0.0433055
[3375]	valid_0's mape: 0.0433047
[3376]	valid_0's mape: 0.0432964
[3377]	valid_0's mape: 0.0432959
[3378]	valid_0's mape: 0.0432997
[3379]	valid_0's mape: 0.0432833
[3380]	valid_0's mape: 0.0432804
[3381]	valid_0's mape: 0.0432736
[3382]	valid_0's mape: 0.0432766
[3383]	valid_0

[3602]	valid_0's mape: 0.0432063
[3603]	valid_0's mape: 0.0432319
[3604]	valid_0's mape: 0.0432579
[3605]	valid_0's mape: 0.0432399
[3606]	valid_0's mape: 0.0432638
[3607]	valid_0's mape: 0.0432893
[3608]	valid_0's mape: 0.0433261
[3609]	valid_0's mape: 0.0432915
[3610]	valid_0's mape: 0.0432638
[3611]	valid_0's mape: 0.043236
[3612]	valid_0's mape: 0.0434067
[3613]	valid_0's mape: 0.0434488
[3614]	valid_0's mape: 0.0434863
[3615]	valid_0's mape: 0.0434198
[3616]	valid_0's mape: 0.043373
[3617]	valid_0's mape: 0.0434122
[3618]	valid_0's mape: 0.0433674
[3619]	valid_0's mape: 0.0433151
[3620]	valid_0's mape: 0.0433414
[3621]	valid_0's mape: 0.0433033
[3622]	valid_0's mape: 0.0433392
[3623]	valid_0's mape: 0.0433801
[3624]	valid_0's mape: 0.0433293
[3625]	valid_0's mape: 0.0432839
[3626]	valid_0's mape: 0.0432519
[3627]	valid_0's mape: 0.0432247
[3628]	valid_0's mape: 0.0432004
[3629]	valid_0's mape: 0.043188
[3630]	valid_0's mape: 0.0432086
[3631]	valid_0's mape: 0.0431828
[3632]	valid_

[3851]	valid_0's mape: 0.0430822
[3852]	valid_0's mape: 0.043107
[3853]	valid_0's mape: 0.0431278
[3854]	valid_0's mape: 0.0431503
[3855]	valid_0's mape: 0.0431162
[3856]	valid_0's mape: 0.0431429
[3857]	valid_0's mape: 0.0431687
[3858]	valid_0's mape: 0.0431377
[3859]	valid_0's mape: 0.0431682
[3860]	valid_0's mape: 0.0431368
[3861]	valid_0's mape: 0.0431094
[3862]	valid_0's mape: 0.0431279
[3863]	valid_0's mape: 0.0431633
[3864]	valid_0's mape: 0.0431366
[3865]	valid_0's mape: 0.043164
[3866]	valid_0's mape: 0.0431937
[3867]	valid_0's mape: 0.0432374
[3868]	valid_0's mape: 0.0432074
[3869]	valid_0's mape: 0.04324
[3870]	valid_0's mape: 0.0432048
[3871]	valid_0's mape: 0.043235
[3872]	valid_0's mape: 0.0431925
[3873]	valid_0's mape: 0.0432337
[3874]	valid_0's mape: 0.0431945
[3875]	valid_0's mape: 0.0432391
[3876]	valid_0's mape: 0.0431895
[3877]	valid_0's mape: 0.0431573
[3878]	valid_0's mape: 0.0431231
[3879]	valid_0's mape: 0.0430972
[3880]	valid_0's mape: 0.0431185
[3881]	valid_0'

In [24]:
Submit_df

,id,MAPE_sum,MAPE_0,MAPE_1,MAPE_2,MAPE_3,MAPE_4,MAPE_mean
0,test_0000,9.565802e+07,1.920012e+07,1.887844e+07,1.907198e+07,1.955291e+07,1.895457e+07,1.913160e+07
1,test_0001,1.316975e+08,2.625389e+07,2.643720e+07,2.623033e+07,2.641846e+07,2.635757e+07,2.633949e+07
2,test_0002,1.252458e+08,2.487626e+07,2.511335e+07,2.520269e+07,2.518002e+07,2.487345e+07,2.504915e+07
3,test_0003,1.378058e+08,2.748760e+07,2.779567e+07,2.745309e+07,2.749049e+07,2.757892e+07,2.756115e+07
4,test_0004,1.326679e+08,2.655224e+07,2.658547e+07,2.639937e+07,2.653466e+07,2.659614e+07,2.653358e+07
5,test_0005,1.234130e+08,2.479726e+07,2.460243e+07,2.467478e+07,2.471050e+07,2.462801e+07,2.468259e+07
6,test_0006,1.324327e+08,2.664594e+07,2.657579e+07,2.634906e+07,2.647443e+07,2.638751e+07,2.648655e+07
7,test_0007,1.259470e+08,2.542737e+07,2.519023e+07,2.511998e+07,2.499197e+07,2.521750e+07,2.518941e+07
8,test_0008,1.312543e+08,2.616818e+07,2.632480e+07,2.615055e+07,2.639815e+07,2.621263e+07,2.625087e+07
9,test_0009,1.469143e+08,2.944338e+07,2.961603e+07,2.931266e+07,2.922510e+07,2.931711e+07,2.938285e+07


In [25]:
Submit_df[['id','MAPE_mean']].to_csv('submit_0812_2.tsv',header=False,index=False,sep ='\t')